In [0]:
!pip install -q six numpy scipy matplotlib scikit-image opencv-python imageio
!pip install -q keras imgaug
!pip install -q git+https://www.github.com/keras-team/keras-contrib.git

In [0]:
import keras
from keras_contrib.callbacks.cyclical_learning_rate import CyclicLR
import imgaug as ia
from imgaug import augmenters as iaa

In [0]:
import numpy as np
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import SGD
from keras.callbacks import *
from keras.models import load_model


# from keras.callbacks import LearningRateScheduler,ModelCheckpoint,EarlyStopping,LambdaCallback
import os,sys,math

In [0]:
import google
colab_dir='./'
file_name='EIP_CIFAR_10'
if hasattr(google,'colab'):
    from google.colab import drive
    drive.mount('/content/gdrive')
    colab_dir='/content/gdrive/My Drive/Colab Notebooks/'
model_file=colab_dir+file_name+'.h5'

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
#Augmentation and resizing
#augment and then concat samples with original
def augment(dataset,flip=0.5,blur=1.0):
    ia.seed(1)
#     seq = iaa.Sequential([iaa.Fliplr(flip),iaa.GaussianBlur(sigma=(0, blur)),iaa.Sometimes(iaa.Crop(percent=(0, 0.1))),
#                          iaa.Sometimes(iaa.Affine(scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
#                                              translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
#                                              rotate=(-45, 45),shear=(-16, 16),order=[0, 1],cval=(0, 255),mode=ia.ALL))])
    seq = iaa.Sequential([
    iaa.Fliplr(0.5),
    iaa.Crop(percent=(0, 0.1)),
    iaa.Sometimes(0.5,
        iaa.GaussianBlur(sigma=(0, 0.5))
    ),
    iaa.ContrastNormalization((0.75, 1.5)),
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
    iaa.Multiply((0.8, 1.2), per_channel=0.2),
    iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
        rotate=(-25, 25),
        shear=(-8, 8))], random_order=True)    
    return seq.augment_images(dataset)

def augmenter(X,y,start=0,end=1):
    na=True
    ln=len(X)
    print('Before augmentation:',X.shape,y.shape)
    start=int(start*ln)
    end=int(end*ln)
    if na:
      X=augment(X)
    else:
      new_X=augment(X)[start:end]
      new_y=y[start:end]
      X=np.concatenate((X,new_X))
      y=np.concatenate((y,new_y))
    print('After augmentation:',X.shape,y.shape)
    return (X,y)

#26x26 is almost half of 32x32. 22x22 maybe too small even though its exact half.
def resize_imgs(imgs,shape=(26,26)):
    seq = iaa.Sequential([iaa.Scale({"height": shape[0], "width": shape[1]})])
    return seq.augment_images(imgs)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2,num_layers=12):
    global compression
    temp = input
    for _ in range(num_layers):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3),kernel_initializer='he_normal', use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
            Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])        
        temp = concat        
    return temp

def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1),kernel_initializer='he_normal', use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
        Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)    
    return avg

def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)    
    return output
  
def dense_unit(input_layer,num_filter, dropout_rate,num_layers):
    dense_block=add_denseblock(input_layer, num_filter, dropout_rate,num_layers)
    transition_block=add_transition(dense_block, num_filter, dropout_rate)
    return transition_block
  
def dense_units_chain(n,input_layer,num_filter, dropout_rate,num_layers):
    dense_unit_=input_layer
    for i in range(n):
        dense_unit_=dense_unit(dense_unit_,num_filter, dropout_rate,num_layers)
    return dense_unit_

In [0]:
def load_prev_model(model_small=None):
#     print(os.popen('du -sh '+colab_dir+'*').read())
    last_best=os.popen('du -sh '+colab_dir+r'*|sed -r "s/^[0-9\.]+[MK]?\s(.*)$/\1/g;"|tail -1').read().strip()
    model_prev=None
    try:
        print('Attempting to load last best model from file - ',end='')
        last_best_fn=last_best.split('/')[-1]
        last_best_epoch=last_best_fn.split('.')[1].split('-')[0]
        model_prev=load_model(last_best)
        print('Sucess\nLoaded '+last_best_fn, 'epoch :', last_best_epoch)
    except Exception as e:
        print('Failed!\n',e)
        try:
            print('Attempting to load last saved model from file - ',end='')
            model_prev = load_model(model_file)
            print('Sucess\nLoaded model from file',model_file)
        except Exception as e:
    #         print(str(e), 'at line ', sys.exc_info()[2].tb_lineno)
            print('Failed!\n',e)
            try:
                print('Attempting to load in memory, small model - ',end='')
                if len(model_small.layers)>1:
                    model_prev=model_small
                print('Sucess')
            except Exception as e:
                print('Failed!\n',e)
    return model_prev

In [0]:
def copy_weights(model_to,model_from):
    #model_to.set_weights(model_from.get_weights())
    s,err=0,0
    print('Trying to copy weights')
    try:
        if len(model_to.layers) >1 and len(model_from.layers) >1:
            pass
    except Exception as e:
        print('Inavlid models',model_to,model_from)
        return
    for new_layer, layer in zip(model_to.layers[1:], model_from.layers[1:]):
        s+=1
        try:
            new_layer.set_weights(layer.get_weights())
        except Exception as e:
            pass
    print('Done: errors:',err)
      
#new model with larger i/p layer
def larger_model(src_model,shape=(32,32,3)):
    new_input=Input(shape)
    src_model.layers.pop(0)
    new_output=src_model(new_input)
    new_model=Model(new_input,new_output)
    return new_model

# Load CIFAR10 Data
def load_data(resize=False,shape=(26,26),test_augment=False):
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    
    if resize:
        x_train=resize_imgs(x_train,shape)
        x_test=resize_imgs(x_test,shape)
    
    (x_train, y_train) = augmenter(x_train, y_train,end=1)
    if test_augment:
        (x_test, y_test) = augmenter(x_test, y_test,end=1)
    return (x_train, y_train,x_test, y_test)
#create a dnn model
def create_model(input_shape,num_layers,input=None):
    print('Creating model with input shape',input_shape)
    if input is None:
        input = Input(input_shape)
    First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)
    hidden_dense_blocks = dense_units_chain(n_dense_blocks,First_Conv2D,num_filter,dropout_rate,num_layers)
    Last_Block = add_denseblock(hidden_dense_blocks, num_filter, dropout_rate)
    output = output_layer(Last_Block)
    model = Model(inputs=[input], outputs=[output])
    return model

In [0]:
print('====================HYPER PARAMETERS====================')

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
max_epochs = 250
# model_epochs=[[0,15],[0,20],[30,80],[80,100]]
model_epochs=[[0,15],[0,50],[50,80]]
num_layers = 12
# layers_small=12
# layers_large=num_layers-layers_small
layers_large=layers_small=num_layers
num_filter = 12
compression = 0
dropout_rate = 0.2
n_dense_blocks = 3
smaller_input=(26,26,3)
hist=[]


In [0]:
print('====================BEGIN OF SMALLER MODEL====================')

In [0]:
# #load data
# x_train, y_train, x_test, y_test = load_data(resize=True,shape=smaller_input[:-1])

# #callbacks
# model_checkpointer=ModelCheckpoint('weights/small_weights.{epoch:02d}-{val_acc:.2f}.h5', monitor='val_acc',verbose=1, save_best_only=True, save_weights_only=False, mode='max', period=2)

# clr = CyclicLR(base_lr=0.1, max_lr=0.2,step_size=8*(len(y_train)/batch_size))

# # callbacks = snapshot.get_callbacks(model_prefix=model_prefix)
# callbacks = [clr, model_checkpointer]

In [0]:
# #create model
# model_small = create_model((x_train.shape[1:]),num_layers=layers_small)
# model_small.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer=SGD(lr=0.1, decay=1e-4, momentum=0.9, nesterov=True))
# model_small.summary()

In [0]:
# #train model
# h=model_small.fit(x_train, y_train, batch_size=batch_size,verbose=1,initial_epoch=model_epochs[0][0],epochs=model_epochs[0][1], callbacks=callbacks,validation_data=(x_test, y_test))
# hist.append(h)
# model_small.save(model_file)
# print('Saved model_small to disk')

In [0]:
print('====================END OF SMALLER MODEL====================')

In [0]:
print('====================BEGIN OF LARGER MODEL====================')

In [0]:
#load data
x_train, y_train, x_test, y_test = load_data(resize=False)
#callbacks
model_checkpointer=ModelCheckpoint('weights/large_weights.{epoch:02d}-{val_acc:.2f}.h5', monitor='val_acc',verbose=1, save_best_only=True, save_weights_only=False, mode='max', period=2)
clr = CyclicLR(base_lr=0.1, max_lr=0.3,step_size=8*(len(y_train)/batch_size),mode='exp_range',gamma=0.99994)

callbacks = [clr, model_checkpointer]

In [0]:
#create model
# model_prev=load_prev_model(model_small)
model_large1 = create_model(x_train.shape[1:],num_layers=layers_large)
# copy_weights(model_large,model_prev)
model_large1.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer=SGD(lr=0.1, decay=1e-4, momentum=0.9, nesterov=True))
model_large1.summary()

In [0]:
#train model
h=model_large1.fit(x_train, y_train, batch_size=batch_size, verbose=1,initial_epoch=model_epochs[1][0],epochs=model_epochs[1][1], callbacks=callbacks, validation_data=(x_test, y_test))
hist.append(h)
model_large.save(model_file)
print("Saved model to disk")

In [0]:
# dropout_rate = 0.2

# model_large2=create_model(x_train.shape[1:],num_layers=layers_large)
# model_large2.set_weights(model_large1.get_weights())
# del model_large1
#callbacks
model_large2=model_large1
model_checkpointer=ModelCheckpoint('weights/large_weights.{epoch:02d}-{val_acc:.2f}.h5', monitor='val_acc',verbose=1, save_best_only=True, save_weights_only=False, mode='max', period=2)
model_large2.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer=SGD(lr=0.01, decay=1e-4, momentum=0.9, nesterov=True))

callbacks = [ model_checkpointer]

#train model
h=model_large2.fit(x_train, y_train, batch_size=batch_size, verbose=1,initial_epoch=model_epochs[2][0],epochs=model_epochs[2][1], callbacks=callbacks,validation_data=(x_test, y_test))
hist.append(h)
model_large.save(model_file)
print("Saved model to disk")

In [0]:
# model_checkpointer=ModelCheckpoint('large_weights.{epoch:02d}-{val_acc:.2f}.h5', monitor='val_acc',verbose=1, save_best_only=True, save_weights_only=False, mode='max', period=2)
# clr = CyclicLR(base_lr=0.001, max_lr=0.006,step_size=6*(len(y_train)/batch_size),mode='triangular')

# callbacks = [clr, model_checkpointer]
# model_large_2=load_model('large_weights.112-0.87.h5')

In [0]:
# #train model

# h=model_large_2.fit(x_train, y_train, batch_size=batch_size, verbose=1,initial_epoch=model_epochs[3][0],epochs=model_epochs[3][1], callbacks=callbacks,validation_data=(x_test, y_test))
# hist.append(h)
# model_large.save(model_file)
# print("Saved model to disk")

In [0]:
# Test the model
score = model_large2.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [0]:
print('====================END OF LARGER MODEL====================')